In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
nltk.download('stopwords')
nltk.download('punkt')
import re
def harfdegistir(cumle):
    listcumle0 = list(cumle)
    for i in range(0,len(listcumle0)):
        if listcumle0[i] == "ü":
            listcumle0[i] = "u"
        elif listcumle0[i] == "Ü":
            listcumle0[i] = "U"
        elif listcumle0[i] == "ö":
            listcumle0[i] = "o"
        elif listcumle0[i] == "Ö":
            listcumle0[i] = "O"
        elif listcumle0[i] == "İ":
            listcumle0[i] = "I"
        elif listcumle0[i] == "ç":
            listcumle0[i] = "c"
        elif listcumle0[i] == "Ç":
            listcumle0[i] = "C"
        elif listcumle0[i] == "ğ":
            listcumle0[i] = "g"
        elif listcumle0[i] == "Ğ":
            listcumle0[i] = "G"
        elif listcumle0[i] == "ş":
            listcumle0[i] = "s"
        elif listcumle0[i] == "Ş":
            listcumle0[i] = "S"
        elif listcumle0[i] == "ı":
            listcumle0[i] = "i"

    cumle0 = ''.join(listcumle0)

    cumle1 = re.sub("[^a-zA-Z]", " ", cumle0)
    cumle2 = cumle1.lower()

    cumle2 = nltk.word_tokenize(cumle2)
    cumle2 = [word for word in cumle2 if not word in set(stopwords.words("turkish"))]

    cumle3 = " ".join(cumle2)

    return cumle3
df = pd.read_csv('yorumlar.csv',encoding="utf-16")
df = df.rename({'Görüş': 'Gorus'}, axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df= df.dropna()
SonGorus = []
X = df["Gorus"].values
for i in range(len(X)):
    X_ = harfdegistir(X[i])
    SonGorus.append(X_)

In [11]:
df['Gorus'] = SonGorus

<ipython-input-11-b586e09cbd7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gorus'] = SonGorus


In [12]:
df

,Gorus,Durum
0,ses kalitesi ergonomisi rezalet sony oldugu ic...,Olumsuz
1,hizli teslimat tesekkurler,Tarafsız
2,ses olayi super gece calistir sikinti yok kabl...,Olumlu
3,geldi bigun kullandik hemen bozoldu hictavsiye...,Olumsuz
4,kulakligin sesi kaliteli falan degil aleti ove...,Olumsuz
...,...,...
11424,biraz fazla isiniyor sorumsuz calisiyor,Tarafsız
11425,orijinal urun fakat ambalaj bize ulasana kadar...,Tarafsız
11426,cihazi yildir kullaniyorum cihazi cok begenere...,Olumsuz
11427,bir sene olmadi ayar dugmesine basiyorum ekran...,Tarafsız


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['Gorus'], df['Durum'], test_size=0.2, random_state=42)

In [14]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [15]:
max_words = 1000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [16]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [17]:
max_len = 100  # Cümlenin maksimum uzunluğu
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [18]:
from tensorflow.keras.layers import Bidirectional, Dropout

# Modeli oluştur
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(X_train_pad, y_train_encoded, epochs=10, validation_data=(X_test_pad, y_test_encoded))

Epoch 1/10
286/286 [==============================] - 227s 760ms/step - loss: 0.8264 - accuracy: 0.6244 - val_loss: 0.7308 - val_accuracy: 0.6780
Epoch 2/10
286/286 [==============================] - 199s 698ms/step - loss: 0.6688 - accuracy: 0.7151 - val_loss: 0.7048 - val_accuracy: 0.6899
Epoch 3/10
286/286 [==============================] - 192s 673ms/step - loss: 0.6198 - accuracy: 0.7433 - val_loss: 0.7023 - val_accuracy: 0.6842
Epoch 4/10
286/286 [==============================] - 199s 697ms/step - loss: 0.5843 - accuracy: 0.7620 - val_loss: 0.7211 - val_accuracy: 0.6877
Epoch 5/10
286/286 [==============================] - 199s 694ms/step - loss: 0.5638 - accuracy: 0.7711 - val_loss: 0.7805 - val_accuracy: 0.6829
Epoch 6/10
286/286 [==============================] - 200s 700ms/step - loss: 0.5282 - accuracy: 0.7884 - val_loss: 0.7733 - val_accuracy: 0.6745
Epoch 7/10
286/286 [==============================] - 191s 669ms/step - loss: 0.5036 - accuracy: 0.8010 - val_loss: 0.7801 -

In [23]:
hbr = pd.read_csv('cekilen_yorum.csv', encoding='utf-8')
hbr.head()

,Unnamed: 0,comment
0,0,Cinayeti savunduğumuz yok ama yazılanları okuy...
1,1,"Ey idareciler , İDAM ne zaman geri gelecek art..."
2,2,İnsan formatını(fıtratını) bozduğunuz zaman ki...
3,3,Kısasta hayat vardır.
4,4,Avustralya ve avrupa ülkelerinde nafaka olayı...


In [22]:
test_loss, test_acc = model.evaluate(X_test_pad)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

72/72 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Test Loss: 0.0
Test Accuracy: 0.0


In [25]:
SonYorum = []
A = hbr["comment"].values
for i in range(len(A)):
    A_ = harfdegistir(A[i])
    SonYorum.append(A_)

In [26]:
hbr['comment'] = SonYorum

In [27]:
hbr

,Unnamed: 0,comment
0,0,cinayeti savundugumuz yok yazilanlari okuyorum...
1,1,ey idareciler idam zaman geri gelecek artik pe...
2,2,insan formatini fitratini bozdugunuz zaman kim...
3,3,kisasta hayat vardir
4,4,avustralya avrupa ulkelerinde nafaka olayi yok...
5,5,turkiye yuzyili nin devlet tesvikli katilleri ...
6,6,tam bir trajedi kadin cinayetlerine bir cozum ...
7,7,bosandigin zaman artik senin hic bir seyin deg...
8,8,vefat eden kadin icin demiyorum genel durumu s...
9,9,nafaka oduyor sa kadina pozitif ayrimcilik yas...


In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
sample_seq = tokenizer.texts_to_sequences(hbr['comment'])
sample_pad = pad_sequences(sample_seq, maxlen=max_len, padding='post')

In [30]:
from sklearn.preprocessing import LabelEncoder
predictions = model.predict(sample_pad)
predicted_classes = predictions.argmax(axis=-1)
predicted_categories = le.inverse_transform(predicted_classes)
# Tahminleri DataFrame'e ekleyerek kategorize edilmiş veriyi oluşturma
result_df = pd.DataFrame({'İçerik': hbr['comment'], 'Tahmin': predicted_categories})
result_df.to_csv('yorum_tahminler.csv', index=False)
# Elde edilen sonuçları görüntüleme
print(result_df)

1/1 [==============================] - 0s 80ms/step
                                               İçerik    Tahmin
0   cinayeti savundugumuz yok yazilanlari okuyorum...   Olumsuz
1   ey idareciler idam zaman geri gelecek artik pe...   Olumsuz
2   insan formatini fitratini bozdugunuz zaman kim...   Olumsuz
3                                kisasta hayat vardir   Olumsuz
4   avustralya avrupa ulkelerinde nafaka olayi yok...  Tarafsız
5   turkiye yuzyili nin devlet tesvikli katilleri ...   Olumsuz
6   tam bir trajedi kadin cinayetlerine bir cozum ...   Olumsuz
7   bosandigin zaman artik senin hic bir seyin deg...  Tarafsız
8   vefat eden kadin icin demiyorum genel durumu s...  Tarafsız
9   nafaka oduyor sa kadina pozitif ayrimcilik yas...   Olumsuz
10  kurban bayrami nda hayvan kesilmesini elestire...   Olumsuz
11  cankaya medeni aydin insanlarin yasadigi semt ...   Olumsuz
12  alaka yav imrahor deresi dahil ankara merkezin...   Olumsuz
13  evlenmeninde bosanmaninda dinimizde kaidesi be..